<a href="https://colab.research.google.com/github/Tanveer-heir/Energy/blob/main/Energy_Predict.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import pandas as pd

df = pd.read_excel("ENB2012_data.xlsx")

df.columns = [
    "Relative_Compactness", "Surface_Area", "Wall_Area", "Roof_Area",
    "Overall_Height", "Orientation", "Glazing_Area",
    "Glazing_Area_Distribution", "Heating_Load", "Cooling_Load"
]

X = df.drop(columns=["Heating_Load", "Cooling_Load"])
y = df["Heating_Load"]

In [4]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)


In [5]:
from sklearn.linear_model import LinearRegression

lr = LinearRegression()
lr.fit(X_train, y_train)


LinearRegression()

In [6]:
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import numpy as np

y_pred = lr.predict(X_test)

mae = mean_absolute_error(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)
r2 = r2_score(y_test, y_pred)

print("MAE :", mae)
print("RMSE:", rmse)
print("R²  :", r2)


MAE : 2.1820729961538947
RMSE: 3.025426853510372
R²  : 0.9121846294352439


In [7]:
from sklearn.model_selection import cross_val_score

neg_mse_scores = cross_val_score(
    lr,
    X,
    y,
    cv=5,
    scoring="neg_mean_squared_error"
)

rmse_scores = np.sqrt(-neg_mse_scores)

r2_scores = cross_val_score(
    lr,
    X,
    y,
    cv=5,
    scoring="r2"
)

print("CV RMSE:", rmse_scores)
print("Mean RMSE:", rmse_scores.mean())
print("Std RMSE :", rmse_scores.std())

print("CV R²:", r2_scores)
print("Mean R²:", r2_scores.mean())


CV RMSE: [4.05571888 3.10815458 2.73856363 2.88835365 3.04003712]
Mean RMSE: 3.166165573359303
Std RMSE : 0.4627313766855095
CV R²: [0.79593126 0.89425246 0.92431909 0.9248329  0.91211934]
Mean R²: 0.890291010420111


In [9]:
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline


In [10]:
from sklearn.linear_model import Ridge
from sklearn.model_selection import cross_val_score
import numpy as np

ridge = Pipeline([
    ("scaler", StandardScaler()),
    ("model", Ridge(alpha=1.0))
])

ridge_rmse = np.sqrt(-cross_val_score(
    ridge, X, y, cv=5, scoring="neg_mean_squared_error"
))

ridge_r2 = cross_val_score(
    ridge, X, y, cv=5, scoring="r2"
)

print("Ridge RMSE Mean:", ridge_rmse.mean())
print("Ridge RMSE Std :", ridge_rmse.std())
print("Ridge R² Mean  :", ridge_r2.mean())


Ridge RMSE Mean: 3.1722381784186453
Ridge RMSE Std : 0.4662887601670683
Ridge R² Mean  : 0.8898258326746061


In [11]:
from sklearn.linear_model import Lasso

lasso = Pipeline([
    ("scaler", StandardScaler()),
    ("model", Lasso(alpha=0.05))
])

lasso_rmse = np.sqrt(-cross_val_score(
    lasso, X, y, cv=5, scoring="neg_mean_squared_error"
))

lasso_r2 = cross_val_score(
    lasso, X, y, cv=5, scoring="r2"
)

print("Lasso RMSE Mean:", lasso_rmse.mean())
print("Lasso RMSE Std :", lasso_rmse.std())
print("Lasso R² Mean  :", lasso_r2.mean())


Lasso RMSE Mean: 3.226735516895979
Lasso RMSE Std : 0.46865456374679215
Lasso R² Mean  : 0.8860790105992532


In [12]:
from sklearn.linear_model import ElasticNet

elastic = Pipeline([
    ("scaler", StandardScaler()),
    ("model", ElasticNet(alpha=0.05, l1_ratio=0.5))
])

elastic_rmse = np.sqrt(-cross_val_score(
    elastic, X, y, cv=5, scoring="neg_mean_squared_error"
))

elastic_r2 = cross_val_score(
    elastic, X, y, cv=5, scoring="r2"
)

print("ElasticNet RMSE Mean:", elastic_rmse.mean())
print("ElasticNet RMSE Std :", elastic_rmse.std())
print("ElasticNet R² Mean  :", elastic_r2.mean())


ElasticNet RMSE Mean: 3.272894649298043
ElasticNet RMSE Std : 0.5046612906788657
ElasticNet R² Mean  : 0.8822711242965351


In [13]:
from sklearn.preprocessing import PolynomialFeatures, StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import cross_val_score
import numpy as np


In [14]:
degrees = [1, 2, 3, 4, 5, 6]

results = []

for d in degrees:
    model = Pipeline([
        ("poly", PolynomialFeatures(degree=d, include_bias=False)),
        ("scaler", StandardScaler()),
        ("lr", LinearRegression())
    ])

    rmse = np.sqrt(-cross_val_score(
        model,
        X,
        y,
        cv=5,
        scoring="neg_mean_squared_error"
    ))

    r2 = cross_val_score(
        model,
        X,
        y,
        cv=5,
        scoring="r2"
    )

    results.append((d, rmse.mean(), rmse.std(), r2.mean()))

    print(f"Degree {d} | RMSE: {rmse.mean():.3f} | Std: {rmse.std():.3f} | R²: {r2.mean():.3f}")


Degree 1 | RMSE: 3.166 | Std: 0.463 | R²: 0.890
Degree 2 | RMSE: 1.476 | Std: 0.885 | R²: 0.967
Degree 3 | RMSE: 59.499 | Std: 117.193 | R²: -213.307
Degree 4 | RMSE: 482.648 | Std: 617.387 | R²: -7209.015
Degree 5 | RMSE: 5596.635 | Std: 5077.017 | R²: -628288.295
Degree 6 | RMSE: 43305.619 | Std: 54555.608 | R²: -58867615.236


In [15]:
from sklearn.linear_model import Ridge

poly_ridge = Pipeline([
    ("poly", PolynomialFeatures(degree=5, include_bias=False)),
    ("scaler", StandardScaler()),
    ("ridge", Ridge(alpha=1.0))
])

rmse = np.sqrt(-cross_val_score(
    poly_ridge,
    X,
    y,
    cv=5,
    scoring="neg_mean_squared_error"
))

r2 = cross_val_score(
    poly_ridge,
    X,
    y,
    cv=5,
    scoring="r2"
)

print("Poly Degree 5 + Ridge")
print("RMSE Mean:", rmse.mean())
print("RMSE Std :", rmse.std())
print("R² Mean  :", r2.mean())


Poly Degree 5 + Ridge
RMSE Mean: 2.1728489096771098
RMSE Std : 0.7264615694256276
R² Mean  : 0.941700725603261


In [16]:
from sklearn.svm import SVR
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import cross_val_score
import numpy as np

svr_linear = Pipeline([
    ("scaler", StandardScaler()),
    ("svr", SVR(kernel="linear", C=10, epsilon=0.1))
])

rmse = np.sqrt(-cross_val_score(
    svr_linear,
    X,
    y,
    cv=5,
    scoring="neg_mean_squared_error"
))

r2 = cross_val_score(
    svr_linear,
    X,
    y,
    cv=5,
    scoring="r2"
)

print("Linear SVR")
print("RMSE Mean:", rmse.mean())
print("RMSE Std :", rmse.std())
print("R² Mean  :", r2.mean())


Linear SVR
RMSE Mean: 3.1348744074157486
RMSE Std : 0.525241905032745
R² Mean  : 0.8910355026494388


In [17]:
svr_rbf = Pipeline([
    ("scaler", StandardScaler()),
    ("svr", SVR(kernel="rbf", C=100, gamma="scale", epsilon=0.1))
])

rmse = np.sqrt(-cross_val_score(
    svr_rbf,
    X,
    y,
    cv=5,
    scoring="neg_mean_squared_error"
))

r2 = cross_val_score(
    svr_rbf,
    X,
    y,
    cv=5,
    scoring="r2"
)

print("RBF SVR")
print("RMSE Mean:", rmse.mean())
print("RMSE Std :", rmse.std())
print("R² Mean  :", r2.mean())


RBF SVR
RMSE Mean: 1.6323819486107056
RMSE Std : 0.5827091217102661
R² Mean  : 0.967329947065165


In [18]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import cross_val_score
import numpy as np

rf = RandomForestRegressor(
    n_estimators=300,
    max_depth=None,
    min_samples_leaf=1,
    random_state=42,
    n_jobs=-1
)

rmse = np.sqrt(-cross_val_score(
    rf,
    X,
    y,
    cv=5,
    scoring="neg_mean_squared_error"
))

r2 = cross_val_score(
    rf,
    X,
    y,
    cv=5,
    scoring="r2"
)

print("Random Forest Regressor")
print("RMSE Mean:", rmse.mean())
print("RMSE Std :", rmse.std())
print("R² Mean  :", r2.mean())


Random Forest Regressor
RMSE Mean: 1.2285924866445652
RMSE Std : 1.1746174167382468
R² Mean  : 0.9650242878809976
